In [2]:
import pandas as pd
import numpy as np
import nltk
import re

In [3]:
df = pd.read_csv(r"/home/sreenal/AI/NLP/Restaurant_Reviews.tsv",delimiter='\t',quoting=3)

In [4]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
df.shape

(1000, 2)

In [6]:
stopwords_list = nltk.corpus.stopwords.words("english") # stopwords list for english
tokener = nltk.word_tokenize # to tokenize

def remove_punch(text):
    clean_words = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    return clean_words

df.Review = df.Review.apply(remove_punch)

In [7]:
def remove_whitespace(text):
    cleaned_space = re.sub(' +'," ",text)
    return cleaned_space
df.Review = df.Review.apply(remove_whitespace)

In [8]:
def remove_stopwords(text):
    tok_word = tokener(text)
    clean_words=[]
    for word in tok_word:
        if word in stopwords_list:
            continue
        else: 
            clean_words.append(word)
    words = ' '.join(clean_words)
    return words

df.Review = df.Review.apply(remove_stopwords)

In [9]:
def remove_digits(text):
    clean_words = re.sub(r"[^a-zA-Z]"," ",text.lower())
    return clean_words

df.Review = df.Review.apply(remove_digits)

In [10]:
df.Review = df.Review.apply(remove_whitespace)

In [11]:
lm=nltk.WordNetLemmatizer()

def lemma(text):
    tok_word = tokener(text)
    lemma_word =[]
    # print("token :",tok_word)
    for tok in tok_word:
        lemma_tok = lm.lemmatize(tok)
        lemma_word.append(lemma_tok)
    # print("lemmatization :",lemma_word)
    final_text= " ".join(lemma_word)
    # print("Final text :",final_text)
    return final_text

df.Review = df.Review.apply(lemma)

In [12]:
df.Review

0                                        wow loved place
1                                             crust good
2                                    tasty texture nasty
3      stopped late may bank holiday rick steve recom...
4                             selection menu great price
                             ...                        
995                    think food flavor texture lacking
996                              appetite instantly gone
997                      overall impressed would go back
998    whole experience underwhelming think go ninja ...
999    wasted enough life poured salt wound drawing t...
Name: Review, Length: 1000, dtype: object

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(ngram_range=(1,3),max_features=2500)
scaled_X_tf =  tf_idf.fit_transform(df.Review)


In [14]:
df_tfidf_sklearn = pd.DataFrame(scaled_X_tf.toarray(),columns=tf_idf.get_feature_names())
df_tfidf_sklearn.head()

,absolutely,absolutely amazing,acknowledged,actually,added,ago,almost,also,also taste,although,...,wow,wrap,wrong,year,year ago,yet,yum,yummy,zero,zero star
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.572716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# train Test split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df_tfidf_sklearn,df['Liked'],test_size=0.3,random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf_tfidf = RandomForestClassifier()
clf_tfidf.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
clf_tfidf.score(X_train,y_train)

0.9957142857142857

In [18]:
clf_tfidf.score(X_test,y_test)

0.7433333333333333

In [19]:



def predict_statement(sample_review):
    text = remove_punch(sample_review)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    text = remove_digits(text)
    text = remove_whitespace(text)
    text = lemma(text)
    temp = tf_idf.transform([text]).toarray()
    out = clf_tfidf.predict(temp)
    return out[0]


In [20]:
sample_review = 'Food was pretty bad and the service was very slow.'

if predict_statement(sample_review):
    print("This is a POSITIVE Review...")
else:
    print("This is a NEGATIVE Review...")

This is a NEGATIVE Review...


TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(ngram_range=(1,3),max_features=2500)
scaled_X_tf =  tf_idf.fit_transform(df.Review)



In [22]:
df_tfidf_sklearn = pd.DataFrame(scaled_X_tf.toarray(),columns=tf_idf.get_feature_names())
df_tfidf_sklearn.head()

,absolutely,absolutely amazing,acknowledged,actually,added,ago,almost,also,also taste,although,...,wow,wrap,wrong,year,year ago,yet,yum,yummy,zero,zero star
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.572716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df_tfidf_sklearn,df['Liked'],test_size=0.3,random_state=42)

In [25]:
clf_tfidf = RandomForestClassifier()
clf_tfidf.fit(X_train, y_train)

RandomForestClassifier()

In [26]:
clf_tfidf.score(X_train,y_train)

0.9957142857142857

In [27]:
clf_tfidf.score(X_test,y_test)

0.7566666666666667

In [28]:
def predict_statement(sample_review):
    text = remove_punch(sample_review)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    text = remove_digits(text)
    text = remove_whitespace(text)
    text = lemma(text)
    temp = tf_idf.transform([text]).toarray()
    out = clf_tfidf.predict(temp)
    return out[0]


In [30]:
sample_review = 'Food was pretty good and the service was very humble.'

if predict_statement(sample_review):
    print("This is a POSITIVE Review...")
else:
    print("This is a NEGATIVE Review...")

This is a POSITIVE Review...
